In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow_decision_forests as tfdf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [8]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Visualization 
TODO

In [13]:
summary = pd.DataFrame({
    'Column Type': train_data.dtypes,
    'Missing Values': train_data.isnull().sum()
}).reset_index().rename(columns={'index': 'Column'})

print(summary)

         Column Column Type  Missing Values
0   PassengerId       int64               0
1      Survived       int64               0
2        Pclass       int64               0
3          Name      object               0
4           Sex      object               0
5           Age     float64             177
6         SibSp       int64               0
7         Parch       int64               0
8        Ticket      object               0
9          Fare     float64               0
10        Cabin      object             687
11     Embarked      object               2


In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

# Logistic regression model

In [ ]:
train_data = pd.get_dummies(train_data, columns=["Sex", "SibSp", "Parch"], drop_first=True)
train_data["Age"] = train_data["Age"].fillna('Missing')
print(df_encoded)


In [52]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train_data = train_data.dropna(subset=['Age'])
y = train_data["Survived"]
features = ["Pclass", "Age", "Sex", "SibSp", "Parch"]
X = train_data[features]
X['Sex'] = le.fit_transform(X['Sex'])
test_data['Sex'] = le.transform(test_data['Sex'])

summary = pd.DataFrame({
    'Column Type': X.dtypes,
    'Missing Values': X.isnull().sum()
}).reset_index().rename(columns={'index': 'Column'})

print(summary)

   Column Column Type  Missing Values
0  Pclass       int64               0
1     Age     float64               0
2     Sex       int64               0
3   SibSp       int64               0
4   Parch       int64               0


/tmp/ipykernel_35/1002122242.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'] = le.fit_transform(X['Sex'])


In [30]:
X.head()

,Pclass,Age,Sex,SibSp,Parch
0,3,22.0,1,1,0
1,1,38.0,0,1,0
2,3,26.0,0,0,0
3,1,35.0,0,1,0
4,3,35.0,1,0,0


In [59]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X , y)
clf.score(X, y)

0.8109243697478992

In [62]:
clf.coef_

array([[-1.2732274 , -0.04320613, -2.50923375, -0.34715453, -0.02551447]])

In [46]:
yPred = clf.predict(test_data[features].dropna(subset=['Age']))
outputPred = pd.DataFrame({'PassengerId': test_data.dropna(subset=['Age']).PassengerId, 'Survived': yPred})
outputCh = pd.DataFrame({'PassengerId': test_data[test_data['Age'].isnull()].PassengerId, 'Survived': 0})
pd.concat([outputPred, outputCh]).to_csv('submission.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Age", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=90, max_depth=6, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

# Random Forest Decision Tree

In [16]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="Survived")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [11]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 4 thread(s) for training
Use /tmp/tmpwsyfot9t as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'PassengerId': <tf.Tensor 'data:0' shape=(None,) dtype=int64>, 'Pclass': <tf.Tensor 'data_1:0' shape=(None,) dtype=int64>, 'Name': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'Sex': <tf.Tensor 'data_3:0' shape=(None,) dtype=string>, 'Age': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'SibSp': <tf.Tensor 'data_5:0' shape=(None,) dtype=int64>, 'Parch': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>, 'Ticket': <tf.Tensor 'data_7:0' shape=(None,) dtype=string>, 'Fare': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'Cabin': <tf.Tensor 'data_9:0' shape=(None,) dtype=string>, 'Embarked': <tf.Tensor 'data_10:0' shape=(None,) dtype=string>}
Label: Tensor("data_11:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'PassengerId': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' s

In [12]:
model_1.compile(metrics=["accuracy"])

In [30]:
y_pred = model_1.predict(test_ds)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': None})
output['Survived'] = (y_pred >= 0.5).astype(int)

1/1 [==============================] - 0s 49ms/step


In [31]:
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [32]:
output.to_csv('submission.csv', index=False)

In [13]:
model_1.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	PassengerId
	Pclass
	Sex
	SibSp
	Ticket

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.         "Sex"  0.585232 ################
    2.        "Fare"  0.277046 #####
    3.         "Age"  0.224175 ###
    4.      "Pclass"  0.213240 ###
    5. "PassengerId"  0.178333 ##
    6.      "Ticket"  0.144835 #
    7.    "Embarked"  0.128923 
    8.       "SibSp"  0.116724 
    9.       "Parch"  0.115361 

Variable Importance: NUM_AS_ROOT:
    1.      "Sex" 176.000000 ################
    2.     "Fare" 56.000000 #####
    3.   "Pclass" 56.00000

In [15]:
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0, max_depth=6)